In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import timedelta

GitHubUserURLBase = "https://api.github.com/users/"
GitHubPublicEvents = "/events/public"

class GitHubUser:
    """
    Creates and github user object
    """
    def __init__(self, username):
        self.username = username
    def get_github_user(self):
        """
        method get_github_user gets user details
        """
        UserInfo = requests.get(GitHubUserURLBase + self.username)
        UserInfoUsable = UserInfo.json()
        return(UserInfoUsable)
    def verify_github_user(self):
        """
        method verify_github_user verifies the user exists
        returns a error if user doesn't exist
        """
        UserInfo = self.get_github_user()
        if 'login' in UserInfo:
            return("User_Exists")
        else:
            raise ValueError("Not a valid GitHub user")
    def get_github_user_activity(self):
        """
        returns the public activity for a user
        """
        UserActivityList = []
        PageNumber = 1
        while PageNumber <= 10:
            PageNumberStr = str(PageNumber)
            print("Getting events Page " + PageNumberStr + " of 10")
            UserActivity = requests.get(GitHubUserURLBase + self.username + GitHubPublicEvents + "?page=" + PageNumberStr)
            UserActivityPage = UserActivity.json()
            if len(UserActivityPage) < 30:
                PageNumber += 10
                print("Ohh this user isn't MAX ACTIVE.")
                print("Working with the dataset they have, even though its not MAX ACTIVE.")
            UserActivityList = UserActivityList + UserActivityPage
            PageNumber += 1
            try:
                len(UserActivityPage) >= 1
            except:
                print("This user has no activity in the last 90 days")
            # print(len(UserActivityList))
        # print(len(UserActivityList))
        return(UserActivityList)

# User = GitHubUser("detaer")
# User.get_github_user_activity()
    
def UserActivityFrame(userevents):
    ids = []
    types = []
    times = []
    for activity in userevents:
        ids.append(activity['id'])
        types.append(activity['type'])
        times.append(activity['created_at'])
    df = pd.DataFrame({'id' : ids,
        'type' : types,
        'created_at' : times,
        'hour_of_day' : types,
        'day_of_week' : types})
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['created_at'] = df['created_at'] - timedelta(hours = 8)
    df['hour_of_day'] = df.created_at.dt.hour
    df['day_of_week'] = df.created_at.dt.weekday_name
    return(df)

def GraphDayOfWeek(EventList):
    plt.close('all')
    EventsPerWeekDay = EventList['day_of_week'].value_counts().sort_index()
    EventsPerWeekDay.index.name = 'Pushes per day'
    EventsPerWeekDayGraph = EventsPerWeekDay.plot(kind='bar', yticks=EventsPerWeekDay)
    EventsPerWeekDayGraph.plot(xtics=EventsPerWeekDayGraph)
    plt.show()
    
def GraphHoursOfDay(EventList):
    plt.close('all')
    EventsPerHourOfDayGraph = EventList['hour_of_day'].value_counts().sort_index()
    EventsPerHourOfDayGraph.index.name = 'Hour of the day 24h format'
    print(list(EventsPerHourOfDayGraph))
    EventsPerHourOfDayGraph.plot(kind='barh', xticks=EventsPerHourOfDayGraph, figsize=(7, 5),label='Pushes that hour.')
    print(EventsPerHourOfDayGraph)
    print(type(EventsPerHourOfDayGraph))
    plt.show()


def main():
    while True:
        UserNameToTest = str(input("Enter the GitHub username of the person you want to creep on:"))
        User = GitHubUser(UserNameToTest)
        ##do a try except to throw an error if the user isn't real, if the user is real, grab their public activity
        try:
            TestUser = User.verify_github_user()
            print(TestUser)
            print("Working!")
        except:
            print("User is not a valid github user!")
            return()
        try:
            UserEvents = User.get_github_user_activity()
        except:
            main()
        UserEventsDF = UserActivityFrame(UserEvents)
        MenuItem = int(input(f"Would you like to graph\n 1) What hours {UserNameToTest} pushes code \n 2) What Day of the week {UserNameToTest} pushes code \n Or any other character to quit \n Go on, graph a thing:"))
        if MenuItem == 1:
            print(f"\nWhat hours of the day is {UserNameToTest} pushing code?")
            GraphHoursOfDay(UserEventsDF)
            return()
        elif MenuItem == 2:
            print(f"Day of the week does {UserNameToTest} push code?")
            GraphDayOfWeek(UserEventsDF)
            return()
        else:
            print("I am starting to doubt your commitment to Sparkle Motion")
            print("What was that... This is not a cherade. We need total concentration. Now, this time, with feeling.")
            main()

if __name__ == '__main__':
    main()
